## Generate polygenic risk score

This notebook is used to generate a polygenic risk score in UKB training set (approx 380k european subjects). This is then separately validated in 50k unrelated UKB european subjects. This polygenic risk score is constructed using LDPred2 following the tutorial at https://privefl.github.io/bigsnpr/articles/LDpred2.html.

This script is implemented in R.

### Prerequisites

In [1]:
library(bigsnpr)
library(data.table)

options(bigstatsr.check.parallel.blas = FALSE)
options(default.nproc.blas = NULL)

Loading required package: bigstatsr



### Import summary statistics

Latent trait GWAS (BOLT-LMM, then GenomicSEM) on UKB training sets.

In [ ]:
gwas_cystatin = read.table('/mnt/grid/ukbiobank/data/Application58510/skleeman/gwas_cystatinc/PRS/EUR/summ_SEM_cystatin_vaf_effectflip_exome.tsv', sep = '\t', header = TRUE)

#Set column labels as per LDPred2 instructions
names(gwas_cystatin) = c('chr','pos','SNPID','a1','a0','MAF','beta','beta_se','P','N','ALT_FREQ')

### Match SNPs to LD reference

LD reference downloaded per LDPred2 tutorial

In [ ]:
map_ldref <- readRDS("ukb_ld/map.rds")
sumstats <- gwas_cystatin
sumstats$n_eff <- sumstats$N

info_snp <- snp_match(sumstats, map_ldref, match.min.prop=0)
(info_snp <- tidyr::drop_na(tibble::as_tibble(info_snp)))

df_beta <- info_snp

### Generate LD matrices

In [ ]:
tmp <- tempfile(tmpdir = "tmp-data")

for (chr in 1:22) {

  cat(chr, ".. ", sep = "")

  ## indices in 'df_beta'
  ind.chr <- which(df_beta$chr == chr)
  ## indices in 'map_ldref'
  ind.chr2 <- df_beta$`_NUM_ID_`[ind.chr]
  ## indices in 'corr_chr'
  ind.chr3 <- match(ind.chr2, which(map_ldref$chr == chr))

  corr_chr <- readRDS(paste0("ukb_ld/LD_chr", chr, ".rds"))[ind.chr3, ind.chr3]

  if (chr == 1) {
    corr <- as_SFBM(corr_chr, tmp)
  } else {
    corr$add_columns(corr_chr, nrow(corr))
  }
}

saveRDS(corr, file = "corr_hm3_cystatin_prs.rds")
saveRDS(df_beta, file = "df_beta_hm3_cystatin_prs.rds")

### Run LDPred2-AUTO

Difficult to perform LDSC using a latent trait so we use a placeholder heritability estimate of 0.4, this appeared to perform well for our use case.

In [ ]:
map_ldref <- readRDS("ukb_ld/map.rds")

N_CORES = 32
corr = readRDS(file = "corr_hm3_cystatin_prs_exome.rds")
df_beta = readRDS(file = "df_beta_hm3_cystatin_prs_exome.rds")

h2_est <- 0.4

# LDpred2-auto
multi_auto <- snp_ldpred2_auto(corr, df_beta, h2_init = h2_est,
                               vec_p_init = seq_log(1e-4, 0.9, N_CORES),
                               ncores = N_CORES)
beta_auto <- sapply(multi_auto, function(auto) auto$beta_est)
                    
saveRDS(multi_auto, file = "multi_auto_hm3_cystatin_prs.rds")
saveRDS(beta_auto, file = "beta_auto_hm3_cystatin_prs.rds")

### Visualize chain

Authors recommend visually inspected the trace to confirm that the model has converged, in our case it clearly does.

In [ ]:
multi_auto = readRDS(file = "multi_auto_hm3_cystatin_prs.rds")
beta_auto = readRDS(file = "beta_auto_hm3_cystatin_prs.rds")
df_beta = readRDS(file = "df_beta_hm3_cystatin_prs.rds")

library(ggplot2)
auto <- multi_auto[[1]]
plot_grid(
  qplot(y = auto$path_p_est) + 
    theme_bigstatsr() + 
    geom_hline(yintercept = auto$p_est, col = "blue") +
    scale_y_log10() +
    labs(y = "p"),
  qplot(y = auto$path_h2_est) + 
    theme_bigstatsr() + 
    geom_hline(yintercept = auto$h2_est, col = "blue") +
    labs(y = "h2"),
  ncol = 1, align = "hv"
)

### Generate score

We create a variant column as an alternative to match by dbSNP ID, this is helpful for the exome sequencing cohort as saves us having to annotate each SNP with dbSNP ID.

In [ ]:
info_snp = df_beta

info_snp$beta_auto <- rowMeans(beta_auto)
info_snp$variant <- paste(info_snp$chr, info_snp$pos, sep=':')
save = info_snp[,c(5, 4, 21, 22)]
write.table(save, file='UKB380_PGS_LDPRED2.tsv', quote=FALSE, sep='\t', row.names = FALSE)